In [ ]:
# !pip install OpenDartReader

In [2]:
from financial_data_making import financial_dict
import OpenDartReader
import pandas as pd

# df = pd.read_csv('./merged_code.csv')
call =  financial_dict('1ee244d6168bde153cb2d45463d71139f567d1f4') # api_key

df = pd.read_csv('esg_finance_data.csv', dtype={'corp_code': 'str', 'stock_code': 'str'})

df['year'] = ['2020'] * 847 + ['2021'] * 847 + ['2022'] * 847
df['year'] = df['year'].astype('int')
print(df.dtypes)
print(df.shape)
# df.to_csv('final_corp_info.csv')
df.head(3)

corp_name     object
esg           object
fin_state     object
corp_code     object
stock_code    object
year           int32
dtype: object
(2541, 6)


,corp_name,esg,fin_state,corp_code,stock_code,year
0,기아,A+,"60,490,443,000,000",00106641,000270,2020
1,S-Oil,A+,"15,690,510,000,000",00138279,010950,2020
2,풀무원,A+,"1,693,573,176,270",00155355,017810,2020


In [3]:
api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
dart = OpenDartReader(api_key)
code, year = '311690', 2020

# 매출액
# 손익계산서
def sales_docu(code, year):
    sales = dart.finstate(code, year)
    # real_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')].iloc[0, :] # 매출액 
    c_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['thstrm_amount'].values # 매출액 # 문제1
    p_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['frmtrm_amount'].values # t-1기 매출액 # 문제 2

    div_cash = dart.report(code, '배당', year) #(연결)현금배당성향(%)
    c_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['thstrm'].values[0] # 순이익이 마이너스일 경우 0% # 배당금 / 당기순이익
    p_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['frmtrm'].values[0]

    holder = dart.report(code, '최대주주', year) # 보통주 계 + 우선수 계 (%)
    ratio_stakeholder = holder[(holder['stock_knd']=='보통주')&(holder['nm']=='계')].values # 문제3

    exec = dart.report(code, '임원', year) #'사외이사' / 전체 임원 수   
    out_exec = exec[exec['ofcps'] == '사외이사'].shape[0]
    exec = exec.shape[0]

    employee = dart.report(code, '직원', year)['rgllbr_co'].to_list() # 총 직원 수
    
    output = {'sales': c_sales, 
              'pre_sales': p_sales, 
              'cash_div': c_div, 
              'pre_cash_civ': p_div,
              'stakeholder' : ratio_stakeholder,
              'num_outexecutives': out_exec,
              'num_executives': exec,
              'num_employee': employee}
    
    return output

In [3]:
# 데이터 수집 (1)

data = []
good_index = []
bad_index = []

for index, (code , year) in enumerate(zip(df['stock_code'], df['year'])):
    try:
        data.append(sales_docu(code, year))
        good_index.append(index)
    except:
        bad_index.append(index)
print(len(good_index), len(bad_index)) # 

# data downloading
with open ('s_docu.txt', 'w', encoding='utf-8') as f:
    for x in df.iloc[good_index, 0].values:
        f.write(f'{x}\n')
    f.close()
    
with open ('f_docu.txt', 'w', encoding='utf-8') as f:
    for x in df.iloc[bad_index, 0].values:
        f.write(f'{x}\n')
    f.close()

In [10]:
api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
dart = OpenDartReader(api_key)

def report_docu(code, year):
    lst = dart.list(code, start='2020', end='2023') # 2020-2023 자료 중에 데이터 찾기
    
    # 변경1: 모든 사업 보고서에서 원하는 년도의 사업 보고서 고르기 
    report_num = lst[(lst['report_nm'].str.contains('사업보고서'))&(lst['report_nm'].str.contains(str(year)))].iloc[0, 5]
    url = dart.sub_docs(report_num) #사업보고서
    html = url[url['title'].str.contains('연결재무제표')].iloc[0, 1]

    docu = pd.read_html(html)
    if docu[1].empty | docu[3].empty:
        print('Some document cannot be extracted')
    else:
        fin_state = docu[1].rename(columns={'Unnamed: 0': 'tag'}) # 재무상태표

        asset = fin_state[fin_state['tag'] == '자산총계'].values # 원래는 iloc[0, 1] # 자산 총계
        debt= fin_state[fin_state['tag'] == '부채총계'].values # 원래는 iloc[0, 1] # 부채 총계
        intangible = fin_state[fin_state['tag'] == '무형자산'].values # 원래는 iloc[0, 1] # 무형자산 

        income = docu[3].rename(columns={'Unnamed: 0': 'tag'}) # 손익계산서

        profit = income[income['tag'].str.contains('영업이익')].values # 원래는 iloc[0, 1] # ()는 마이너스를 뜻함 # 영업이익(손실)
        c_profit_tax = income[income['tag'].str.contains('법인세비용차감전')].values # 원래는 iloc[0, 1] # t기 법인세비용차감전순이익(손실)
        p_profit_tax = income[income['tag'].str.contains('법인세비용차감전')].values # t-1기 법인세비용차감전순이익(손실)
        
        dict_2 = {'asset': asset,
                    'debt': debt,
                    'intang': intangible,
                    'profit': profit,
                    'profit_without_tax': c_profit_tax,
                    'pre_profit_without_tax': p_profit_tax}   
    return dict_2                                                                                                          

In [12]:
print(report_docu('034730', 2020))

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [5]:
df.head()

,corp_name,esg,fin_state,corp_code,stock_code,year
0,기아,A+,"60,490,443,000,000",00106641,000270,2020
1,S-Oil,A+,"15,690,510,000,000",00138279,010950,2020
2,풀무원,A+,"1,693,573,176,270",00155355,017810,2020
3,케이티,A+,"33,662,545,000,000",00190321,030200,2020
4,SK,A+,"137,667,906,000,000",00181712,034730,2020


In [9]:
lst = dart.list('034730', start='2020', end='2023')
report_num = lst[(lst['report_nm'].str.contains('사업보고서'))&(lst['report_nm'].str.contains(str(2020)))].iloc[0, 5]#'rcept_no']
report_num

'20210319001179'

In [4]:
# 데이터 수집 (2)

data_re = []
good_index_re = []
bad_index_re = []

for index, (code , year) in enumerate(zip(df['stock_code'], df['year'])):
    try:
        data_re.append(report_docu(code, str(year)))
        print(f'{index}, Good!')
        good_index_re.append(index)
    except:
        print(f'{index}, bad...')
        bad_index_re.append(index)
        
print(len(good_index_re), len(bad_index_re)) # 


0, bad...
1, bad...
2, bad...
3, bad...
4, bad...
5, bad...
6, bad...
7, bad...
8, bad...
9, bad...
10, bad...
11, bad...
12, bad...
13, bad...
14, bad...
15, bad...
16, bad...
17, bad...
18, bad...
19, bad...
20, bad...
21, bad...
22, bad...
23, bad...
24, bad...
25, bad...
26, bad...
27, bad...
28, bad...
29, bad...
30, bad...
31, bad...
32, bad...
33, bad...
34, bad...
35, bad...
36, bad...
37, bad...
38, bad...
39, bad...
40, bad...
41, bad...
42, bad...
43, bad...
44, bad...
45, bad...
46, bad...
47, bad...
48, bad...
49, bad...
50, bad...
51, bad...
52, bad...
53, bad...
54, bad...
55, bad...
56, bad...
57, bad...
58, bad...
59, bad...
60, bad...
61, bad...
62, bad...
63, bad...
64, bad...
65, bad...
66, bad...
67, bad...
68, bad...
69, bad...
70, bad...
71, bad...
72, bad...
73, bad...
74, bad...
75, bad...
76, bad...
77, bad...
78, bad...
79, bad...
80, bad...
81, bad...
82, bad...
83, bad...
84, bad...
85, bad...
86, bad...
87, bad...
88, bad...
89, bad...
90, bad...
91, bad..

KeyboardInterrupt: 

In [ ]:
succees_report_docu = df.iloc[good_index_re, 0]
fail_report_docu = df.iloc[bad_index_re, 0]

# data downloading
with open ('s_report.txt', 'w') as f:
    for x in df.iloc[good_index_re, 0].values:
        f.write(f'{x}\n')
    f.close()
    
with open ('f_report.txt', 'w') as f:
    for x in df.iloc[bad_index_re, 0].values:
        f.write(f'{x}\n')
    f.close()
    
import pickle
with open('data_report.pickle', 'wb') as f:
    # for  data in data_re
    pickle.dump(data_re, f)

In [ ]:
with open('data_report.pickle', 'rb') as f:
    loaded = pickle.load(f)
print(loaded)
print(len(loaded))